# Test de création d'isochrones avec de nouvelles formes

On se base sur l'API de Navitia pour obtenir les points qui permettent de composer les isochrones

In [1]:
# Add module to path
import os
import sys

add_path = os.getcwd().replace('experiments', '')
sys.path.append(add_path)

In [2]:
from dotenv import load_dotenv
import docopt
import json
from pathlib import Path
from csv_to_json import csv_to_json
from isos_and_intersections import GetIso

param_csv = os.path.join(add_path, "params/test_journeys.csv")
param_json = os.path.join(add_path, "params/test_journeys.json")
places_cache = os.path.join(add_path, "params/places_cache.json")

#Parameters
env_path = Path(add_path) / '.env'
load_dotenv(dotenv_path=env_path)
TOKEN = os.getenv("NAVITIA_TOKEN")

columns_with_array_of_str = [
    "colors_iso",
    "addresses",
    "excluded_modes",
    "durations"
    ]

#JSON INPUT
json_file = csv_to_json(param_csv, param_json, ";", columns_with_array_of_str)

gdf_global = GetIso(param_json, places_cache, api="navitia", token=TOKEN).get_all_isos()

In [5]:
gdf_global

,arrival_date_time,departure_date_time,duration,geometry,lat_from,lon_from,nb_transfers,requested_date_time,to_id,to_name
0,20181115T080400,20181115T080000,240,POINT (2.349002 48.863713),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:59261,Etienne Marcel
1,20181115T080400,20181115T080000,240,POINT (2.347884 48.858303),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:59627,Châtelet
2,20181115T080500,20181115T080000,300,POINT (2.35264 48.866049),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:59625,Réaumur-Sébastopol
3,20181115T080600,20181115T080000,360,POINT (2.346222 48.855688),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:59628,Cité
4,20181115T080600,20181115T080000,360,POINT (2.329927 48.872157),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:8775859:810:A,Auber
5,20181115T080600,20181115T080000,360,POINT (2.346035 48.853336),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:8778543:810:B,Saint-Michel-Notre-Dame
6,20181115T080700,20181115T080000,420,POINT (2.346495 48.862015),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:59208,Les Halles
7,20181115T080700,20181115T080000,420,POINT (2.354521 48.869635),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:59253,Strasbourg-Saint-Denis
8,20181115T080700,20181115T080000,420,POINT (2.344015 48.853603),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:59255,Saint-Michel
9,20181115T080700,20181115T080000,420,POINT (2.37326 48.844139),48.8621487,2.3467922,0,20181115T080000,stop_point:OIF:SP:8775858:810:A,Gare de Lyon RER A
